In [ ]:
import numpy as np
import torch
import scanpy as sc
import matplotlib.pyplot as plt
#import torchmetrics
from torchmetrics.utilities.data import dim_zero_sum 
import sys
import os
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
print(sys.executable)
#import scarches
import anndata
import pandas as pd
import scvi

In [ ]:
pre = "MH48"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                              pre = pre,
                                                                              add_path = True)

In [ ]:
base_model_path_MH46,_,_,base_anndata_objects_MH46 = h.return_local_paths(drive = drive,
                                                                              pre = 'MH46',
                                                                              add_path = False)

In [ ]:
test_adata = sc.read_h5ad(os.path.join(base_anndata_objects_MH46,'MH46_test_adata_hyperparameter.h5ad'))

In [ ]:
n_latents = [6,8,10,12]
batch_sizes = [8,16,32,64]

In [ ]:
df_eval = pd.DataFrame(columns = ['tool','n_latent','batch_size','MSE_test'])
X = test_adata.X.toarray()
for n_latent in n_latents:
    for batch_size in batch_sizes:
        load_string = os.path.join(
            base_model_path_MH46,'MH46_LDVAE_model_nlat' + str(n_latent) + '_bs' + str(batch_size)  + '.pt')
        model = scvi.model.LinearSCVI.load(load_string)
        reconstructed_X = model.get_normalized_expression(adata = test_adata).values
        mse = np.mean((X - reconstructed_X) ** 2)
        row_to_add = pd.DataFrame(['LDVAE',n_latent,batch_size,mse],index = ['tool','n_latent','batch_size','MSE_test']).T
        df_eval = df_eval.append(row_to_add) 

In [ ]:
df_eval['rank_MSE_test'] = df_eval['MSE_test'].rank(ascending=True).astype(int)
df_eval.to_csv(os.path.join(base_table_path,pre + '_df_eval_hypserach_LDVAE.csv'))

In [ ]:
df_eval

In [ ]:
df_eval[df_eval['rank_MSE_test'] == 1]

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()